In [2]:
import matplotlib.pyplot as plt
%matplotlib inline

from HelmholtzSolver3D import *
from ExampleBoundaries import Sphere

frequency = 20.0 # frequency [Hz]
k = frequencyToWavenumber(frequency)

interiorPoints = np.array([[0.500, 0.000, 0.000],
                           [0.000, 0.000, 0.010],
                           [0.000, 0.000, 0.250],
                           [0.000, 0.000, 0.500],
                           [0.000, 0.000, 0.750]], dtype=np.float32)

# Test Problem 1
# Dirichlet boundary condition with phi = sin(k*z)
#
solver = HelmholtzSolver3D(*(Sphere()))

boundaryCondition = BoundaryCondition(solver.aElement.shape[0])
boundaryCondition.alpha.fill(1.0)
boundaryCondition.beta.fill(0.0)
boundaryCondition.f[:] = np.sin(k * solver.aCenters[:,2])

boundaryIncidence = BoundaryIncidence(solver.aElement.shape[0])
boundaryIncidence.phi.fill(0.0)
boundaryIncidence.v.fill(0.0)

interiorIncidentPhi = np.zeros(interiorPoints.shape[0], dtype=np.complex64)

boundarySolution = solver.solveInteriorBoundary(k, boundaryCondition, boundaryIncidence)

interiorPhi = solver.solveInterior(boundarySolution, interiorIncidentPhi, interiorPoints)

print "Test Problem 1"
print "==============\n"
print boundarySolution
printInteriorSolution(boundarySolution, interiorPhi)


# Test Problem 2
# Neumann boundary condition with v = cos(k*z)
#
boundaryCondition.alpha.fill(0.0)
boundaryCondition.beta.fill(1.0)

for i in range(solver.aElement.shape[0]):
    a = solver.aVertex[solver.aElement[i, 0]]
    b = solver.aVertex[solver.aElement[i, 1]]
    c = solver.aVertex[solver.aElement[i, 2]]
    normal = Normal3D(a, b, c)
    boundaryCondition.f[i] = k * np.cos(k * solver.aCenters[i,2]) * normal[2]

boundarySolution = solver.solveInteriorBoundary(k, boundaryCondition, boundaryIncidence)

interiorPhi = solver.solveInterior(boundarySolution, interiorIncidentPhi, interiorPoints)

print "\n\nTest Problem 2"
print "==============\n"
print boundarySolution
printInteriorSolution(boundarySolution, interiorPhi)


# Test Problem 3
# Neumann boundary condition with v = cos(k*z)
#
boundaryCondition.alpha.fill(0.0)
boundaryCondition.beta.fill(1.0)
boundaryCondition.f.fill(0.0)

p = np.array([0.0, 0.0, 0.0], dtype=np.float32)
for i in range(solver.aElement.shape[0]):
    q = solver.aCenters[i, :]
    r = p - q
    R = norm(r)
    boundaryIncidence.phi[i] = np.exp(1.0j * k * R) / (4.0 * np.pi * R)
    a = solver.aVertex[solver.aElement[i, 0]]
    b = solver.aVertex[solver.aElement[i, 1]]
    c = solver.aVertex[solver.aElement[i, 2]]
    drbdn = -np.dot(r, normal) / R
    boundaryIncidence.v[i] = drbdn * np.exp(1.0j * k * R)*(1.0j * k * R - 1.0) / \
            (4.0 * np.pi * R * R)

for i in range(interiorPoints.shape[0]):
    q = interiorPoints[i, :]
    r = p - q
    R = norm(r)
    interiorIncidentPhi[i] = np.exp(1.0j * k * R) / (4.0 * np.pi * R)

boundarySolution = solver.solveInteriorBoundary(k, boundaryCondition, boundaryIncidence)

interiorPhi = solver.solveInterior(boundarySolution, interiorIncidentPhi, interiorPoints)

print "\n\nTest Problem 3"
print "==============\n"
print boundarySolution
printInteriorSolution(boundarySolution, interiorPhi)

Test Problem 1

Density of medium:      1.205 kg/m^3
Speed of sound:         344.0 m/s
Wavenumber (Frequency): 0.365301471348 (20.0 Hz)

index          Potential                   Pressure                    Velocity              Intensity
    1   2.8039e-01+  0.0000e+00i    0.0000e+00+  4.2459e+01i    3.3805e-01+ -5.1565e-03i    -1.0947e-01
    2   2.8039e-01+  0.0000e+00i    0.0000e+00+  4.2459e+01i    3.3799e-01+ -5.1442e-03i    -1.0921e-01
    3   2.8039e-01+  0.0000e+00i    0.0000e+00+  4.2459e+01i    3.3805e-01+ -5.1565e-03i    -1.0947e-01
    4   2.8039e-01+  0.0000e+00i    0.0000e+00+  4.2459e+01i    3.3805e-01+ -5.1565e-03i    -1.0947e-01
    5   2.8039e-01+  0.0000e+00i    0.0000e+00+  4.2459e+01i    3.3799e-01+ -5.1442e-03i    -1.0921e-01
    6   2.8039e-01+  0.0000e+00i    0.0000e+00+  4.2459e+01i    3.3805e-01+ -5.1565e-03i    -1.0947e-01
    7   1.6172e-01+  0.0000e+00i    0.0000e+00+  2.4489e+01i    1.8979e-01+ -7.3123e-03i    -8.9536e-02
    8   8.1129e-02+  0.0000e+00i

Copyright (C) 2017 Frank Jargstorff

This file is part of the AcousticBEM library.

AcousticBEM is free software: you can redistribute it and/or modify it under the terms of the GNU General Public License as published by the Free Software Foundation, either version 3 of the License, or (at your option) any later version.

AcousticBEM is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the GNU General Public License for more details.

You should have received a copy of the GNU General Public License along with AcousticBEM. If not, see http://www.gnu.org/licenses/.